In [1]:
import pickle
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import re

seed = 2023

In [2]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search, obtain_SWNUNetwork_input
)

In [3]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [4]:
%run load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [11]:
x_data = obtain_SWNUNetwork_input(
    method = "gaussian_random_projection",
    dimension=3,
    df=df_rumours,
    id_column='timeline_id',
    label_column='label',
    embeddings= sbert_embeddings,
    k=5,
    features='time_encoding',
    standardise_method=None,
    add_time_in_path=True,
)

x_data[0].shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


torch.Size([5568, 5, 389])

In [14]:
x_data[1]

4

In [23]:
path = x_data[0][:,:,:4]

In [24]:
path.shape

torch.Size([5568, 5, 4])

`True` values in the mask are the ones that indicate to MultiheadAttention to ignore in attention calculation.

In [34]:
mask = torch.sum(path, 2) == 0

In [33]:
path[0]

tensor([[2.0148e+03, 2.3824e-02, 7.6382e-01, 1.0231e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]], dtype=torch.float64)

In [35]:
mask[0]

tensor([False,  True,  True,  True,  True])

In [36]:
path[1]

tensor([[2.0148e+03, 2.3824e-02, 7.6382e-01, 1.0231e+00],
        [2.0148e+03, 4.0618e-01, 6.7205e-01, 7.2452e-01],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]], dtype=torch.float64)

In [39]:
mask[1]

tensor([False, False,  True,  True,  True])